In [ ]:
!apt-get update && apt-get install -y ffmpeg libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0
!pip install -q \
  gTTS \
  SpeechRecognition \
  playsound \
  PyAudio \
  pyjokes

In [ ]:
import io
import os
import playsound
import pyaudio
import pyjokes
import speech_recognition as sr
import subprocess
import webbrowser

from base64 import b64decode
from datetime import datetime
from google.colab import output
from gtts import gTTS
from IPython.display import HTML, Audio, Javascript
from pygame import mixer

In [ ]:
def open_url(url):
  display(Javascript(f'window.open("{url}");'))

In [ ]:
def text_to_speech(text, language="en", filename="speech.mp3", autoplay=True):
  tts = gTTS(text=text, lang=language, slow=False)
  tts.save(filename)
  return Audio(filename, autoplay=autoplay)

In [ ]:
def respond(text):
    print("Text from get audio " + text)
    if 'youtube' in text:
        open_url("https://www.youtube.com")
    elif 'search' in text:
        open_url("https://www.google.com")
    elif 'piada' in text:
        text_to_speech(pyjokes.get_joke())
    elif 'horas' in text:
        strTime = datetime.today().strftime("%H:%M %p")
        print(strTime)
        text_to_speech(strTime)
    elif 'sair' in text:
        text_to_speech("Adeus, até a próxima.")
        exit()


In [ ]:
def get_text_from_audio(recorded_file, language="pt-BR"):

    try:
      r = sr.Recognizer()
      with sr.AudioFile(recorded_file) as source:
          audio = r.record(source)
      text = r.recognize_google(audio, language=language)  # Use Google Web Speech API
      print("Texto reconhecido:")
      print(text)
      return text

    except ImportError:
      print("Biblioteca SpeechRecognition não encontrada. Por favor, instale-a usando: !pip install SpeechRecognition")
    except Exception as e:
      print(f"Ocorreu um erro durante o reconhecimento de fala: {e}")


In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {  // This is the record function
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream, { mimeType: 'audio/webm; codecs=opus' })
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record_audio(filename, seconds=5):
    print(f"Recording audio for {seconds} seconds...")

    display(Javascript(RECORD))

    s = output.eval_js('record(%d)' % (seconds * 1000))
    b = b64decode(s.split(',')[1])

    webm_filename = "temp_audio.webm"
    with open(webm_filename, "wb") as f:
        f.write(b)

    wav_filename = filename
    try:
        subprocess.run(['ffmpeg', '-i', webm_filename, wav_filename], capture_output=True, check=True)
        print(f"Audio saved to {wav_filename}")
    except subprocess.CalledProcessError as e:
        print(f"Error converting audio: {e}")
        return None
    finally:
      os.remove(webm_filename)

    return wav_filename

In [ ]:
filename = "recorded_audio.wav"
seconds_to_record = 4
recorded_file = record_audio(filename, seconds_to_record)

if recorded_file:
    display(Audio(recorded_file))
    from google.colab import files
    files.download(recorded_file)

    print("Obtendo texto...")

    text = get_text_from_audio(recorded_file)

    respond(text)
